In [1]:
import numpy as np
from typing import List
import torch

from lib.Tensor import Tensor
from lib.NN import Layer

In [5]:
class Dense(Layer):
    def __init__(self, input_dim: int, output_dim: int):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weights = Tensor.randn(output_dim, input_dim) * 0.01 # init backwards (output_dim, input_dim) for computational efficiency
        self.biases = Tensor.zeros(output_dim)

    def forward(self, inputs: Tensor) -> Tensor:
        return inputs @ self.weights.T + self.biases # transpose weights for computational efficiency
    
    def parameters(self) -> List[Tensor]:
        return [self.weights, self.biases]

In [9]:
def test_dense():
    input_dim = 5
    output_dim = 3
    batch_size = 10

    pt_dense = torch.nn.Linear(input_dim, output_dim)
    dense = Dense(input_dim, output_dim)

    # copy the weights and biases from custom layer to pytorch layer
    with torch.no_grad():
        pt_dense.weight.copy_(torch.tensor(dense.weights.data))
        pt_dense.bias.copy_(torch.tensor(dense.biases.data))

    pt_input = torch.randn(batch_size, input_dim) # pytorch tensor
    input = Tensor(pt_input.numpy()) # copy the pytorch tensor

    pt_output = pt_dense(pt_input) # pytorch output
    output = dense(input) # custom output

    # compare the outputs
    assert np.allclose(pt_output.detach().numpy(), output.data, atol=1e-6)

In [10]:
test_dense()

In [11]:
class BatchNorm(Layer):
    def __init__(self, num_features):
        super().__init__()
        self.gamma = Tensor.ones(num_features)
        self.beta = Tensor.zeros(num_features)
        self.eps = 1e-5

    def forward(self, inputs: Tensor) -> Tensor:
        mean = np.mean(inputs.data, axis=0)
        var = np.var(inputs.data, axis=0)
        normalized = (inputs.data - mean) / np.sqrt(var + self.eps)
        out = self.gamma.data * normalized + self.beta.data
        return Tensor(out)

    def parameters(self) -> List[Tensor]:
        return [self.gamma, self.beta]

In [12]:
class Dropout(Layer):
    def __init__(self, p: float = 0.5):
        super().__init__()
        self.p = p

    def forward(self, inputs: Tensor) -> Tensor:
        mask = np.random.binomial(1, 1 - self.p, size=inputs.data.shape)
        out = inputs.data * mask / (1 - self.p)
        return Tensor(out)

    def parameters(self) -> List[Tensor]:
        return []  # Dropout has no learnable parameters

In [17]:
def test_batchnorm():
    num_features = 4
    batch_size = 10

    # Create instances of custom and PyTorch BatchNorm layers
    custom_bn = BatchNorm(num_features)
    pt_bn = torch.nn.BatchNorm1d(num_features, affine=True, eps=1e-5, momentum=0)

    # Initialize both layers with the same gamma and beta
    with torch.no_grad():
        pt_bn.weight.copy_(torch.tensor(custom_bn.gamma.data))
        pt_bn.bias.copy_(torch.tensor(custom_bn.beta.data))

    # Generate random input
    pt_input = torch.randn(batch_size, num_features)
    custom_input = Tensor(pt_input.numpy())

    # Forward pass
    pt_output = pt_bn(pt_input)
    custom_output = custom_bn(custom_input)

    # Check if the outputs are close
    assert np.allclose(pt_output.detach().numpy(), custom_output.data, atol=1e-6)

In [18]:
test_batchnorm()

In [15]:
def test_dropout():
    p = 0.5
    shape = (10, 4)

    # Create custom Dropout layer
    custom_dropout = Dropout(p=p)

    # Generate random input
    custom_input = Tensor(np.random.randn(*shape))

    # Forward pass
    custom_output = custom_dropout(custom_input)

    # Check if the Dropout layer has zeroed approximately p fraction of input
    zero_fraction = np.mean(custom_output.data == 0)
    assert np.isclose(zero_fraction, p, atol=0.1)

In [16]:
test_dropout()